In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import norm, anderson, shapiro
import seaborn as sns; sns.set()
from math import pi,sqrt

from generators import LXM 

# Ejercicio 5
Utilizando el generador implementado en el ejercicio 1:
* Implementar un método para generar variables aleatorias con distribución normal con media 15 y desvío 3.
* Graficar la distribución que siguen los números pseudo aleatorios generados.
* Realizar, al menos, 2 tests de los explicados en la materia para verificar si los números generados siguen la distribución pedida (evalué los resultados para distintos tamaños de muestra).

Empezamos implementando el método para generar las variables aleatorias. Para obtener la normal usamos el método de aceptación-rechazo, es decir, si se obtiene un valor dentro del area esperada, es aceptado, sino, se rechaza.

Una particularidad a tener en cuenta, es que no se puede generar una normal entera debido al rango de la exponencial, por lo que según un nuevo valor random obtenido determinamos de que lado de la normal iría.

In [ ]:
def fY(y):
    return np.exp(-y)

def fX(x):
    return np.exp(-x**2 / 2) / np.sqrt(2 * pi)


def probabilidadAceptar(t, c):
    return fX(t) / (c * fY(t))

def funcionNormal(media, desvio, N):
    lxm = LXM()
    c = np.sqrt(2 * np.exp(1) / pi)
    tExponenciales = np.random.exponential(size = N) 
    
    zNormal = []
    for i in range(N):
        uniforme = lxm.generar(True)
        if uniforme < probabilidadAceptar(tExponenciales[i],c):
            probabilidadSigno = lxm.generar(True)
            if probabilidadSigno < 0.5:
                zNormal.append( tExponenciales[i] * desvio + media)
            else:
                zNormal.append( - tExponenciales[i] * desvio + media)


    return zNormal

vNormal = funcionNormal(15,3,100000)

In [ ]:
plt.hist( vNormal, bins=100)
plt.title('Histograma de Normal')
plt.xlabel('X')
plt.show()

print('Media: ', np.mean(vNormal))
print('Desvio: ', np.std(vNormal))

Vemos de realizar los tests para ver si se sigue la distribución pedida.

Probamos primero aplicando el test de Kolmogorov Smirnov. Este test compara dos distribuciones de probabilidad acumulada, una se asigna a la hipótesis nula y la otra es generada de forma empírica.

Los parámetros que vamos a usar son:
*   Nivel de significación $\alpha$ = 0,05
*   $K_{aceptación}$ tabulado usando K = $ \frac{C}{\sqrt{n}}$ para el nivel de significación C = 0,88



In [ ]:
def Kaceptacion(N):
  return 0.88/(sqrt(N))


def kolmogorovSmirnov(muestra, Kaceptacion):
    kMayor = 0
    kMenor = 0
    i = 1
    n = len(muestra)
    for x in sorted(muestra):
        valorTeorico = norm.cdf(x, loc=15, scale=3)

        kMayor = max(kMayor, (i / n - valorTeorico))
        kMenor = max(kMenor, (valorTeorico - (i -1) / n))
        i += 1

    kMayor = sqrt(n) * kMayor
    kMenor = sqrt(n) * kMenor
    k = max(kMayor, kMenor)
    print("K obtenido = {0}".format(k))
    print("K aceptación = {0}".format(Kaceptacion))
    if  k <= Kaceptacion:
        print('Acepto H0')
    else:
        print('Rechazo H0')



Probamos el test para distintas muestras.

In [ ]:
for muestras in {1000, 10000, 100000, 500000}:
    print("Muestras: {0}".format(muestras))
    vNormal = funcionNormal(15,3,muestras)
    kolmogorovSmirnov(vNormal,Kaceptacion(len(vNormal)))
    print("------")

In [ ]:
def andersonTest(vNormal):
  estadistico, valoresCriticos, nivelDeSignificancia  = anderson(vNormal)

  print("Estadístico: %.3f" % estadistico)

  for i in range(len(valoresCriticos)):
    print("Nivel de significancia: %.3f -- Umbral: %.3f" % (nivelDeSignificancia[i] / 100, valoresCriticos[i]))
    if estadistico < valoresCriticos[i]:
      print("Los datos son normales, no se puede rechazar H0")
    else:
      print("Los datos no son normales, se rechaza H0")

In [ ]:
for muestras in {1000,10000,100000}:
  print("Muestras: {0}".format(muestras))
  v_normal = funcionNormal(15,3,muestras)
  andersonTest(v_normal)
  print("------")
  

Se puede ver que la salida indica que los datos siguen una distribución normal.

Vemos ahora con el test de Shapiro Wilk. Este test determina la normalidad de los datos.

* $H_0$: La muestra sigue la distribución probada.
* $H_1$: La muestra no sigue la distribución probada.

In [ ]:
def shapiroWilkTest(vNormal, nivelSignificacion):
  estadistico, pValor = shapiro(vNormal)

  print("Estadístico: %.3f" % estadistico)
  print("p valor: %.6f" % pValor)
  if pValor < nivelSignificacion:
    print("Los datos no son normales, se rechaza H0")
  else:
    print("No se puede rechazar H0")

In [ ]:
for muestras in {1000,10000,100000}:
    print("Muestras: {0}".format(muestras))
    vNormal = funcionNormal(0,1,muestras)
    shapiroWilkTest(vNormal, 0.05)
    print("------")

Podemos ver que los datos siguen la distribución normal.